In [24]:
#import libraries
import torch
from torch import nn


import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt

# Data preparation and loading

In [25]:
#training and test data

training_data = torchvision.datasets.FashionMNIST( root="data",
                                                  train = True,
                                                  download=True,
                                                  transform=ToTensor(),
                                                  target_transform= None
)

test_data = torchvision.datasets.FashionMNIST(root="data",
                                 train = False,
                                 download=True,
                                 transform = ToTensor(),
                                 
)

In [57]:
#view the returned data, we know data type is tuple
#returns image, index

image, label = training_data[5]


In [ ]:
plt.plot(figsize=(6,7))
plt.imshow(image.squeeze(), cmap='gray')
plt.title(f"{label}")
plt.show()

In [28]:
len(training_data), len(test_data)

(60000, 10000)

In [29]:
image.shape


torch.Size([1, 28, 28])

In [ ]:
class_name = training_data.classes
class_name

In [ ]:
#multiple image plots

fig = plt.figure()
#number of rows and cols
rows, cols = 4, 4

for i in range(1, rows*cols +1):
    random_img = torch.randint(0, len(training_data), size=[1]).item()
    img, label = training_data[random_img]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze())
    plt.axis(False)
    plt.title(class_name[label])



In [37]:
#Dataloader breaking the data to batches for computationaö efficiency
#general mutliples of twos are preferred

training_data_batch = DataLoader(training_data,
                                 batch_size=32,
                                 shuffle=True)

test_data_batch = DataLoader(test_data,
                             batch_size=32,
                             shuffle=True)

print(f"The length of the training batch: {len(training_data_batch)} and test data batch are: {len(test_data_batch)}")

The length of the training batch: 1875 and test data batch are: 313


In [40]:
#the properties of the training batch

training_data_features, training_data_labels = next(iter(training_data_batch))
training_data_features.shape, training_data_labels.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [ ]:
#plot some of the data from the batches
torch.manual_seed(42)

fig = plt.figure(figsize=(9,9))

rows, cols = 4,4

for i in range(1, rows*cols+1):
    random_idx = torch.randint(0, len(training_data_features), size=[1]).item()
    random_img_idx, random_label_idx = training_data_features[random_idx], training_data_labels[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(random_img_idx.squeeze(), cmap='gray')
    plt.axis(False)
    plt.title(f"{class_name[random_label_idx]}")
    


## Building baseline model

In [54]:
class FashionMNISTModel(nn.Module):
    def __init__(self, input_shape: int, hidden_layer: int, output_shape: int):
        super().__init__()
        self.layer_one = nn.Sequential(
                                        nn.Flatten(),
                                        nn.Linear(in_features=input_shape, out_features=hidden_layer),
                                        nn.Linear(in_features=hidden_layer, out_features=output_shape),                                        
        )

    ##define forward method

    def forward(self, x):
        return self.layer_one(x)
 

In [56]:
#create object of the class
model_0 = FashionMNISTModel(784, 10, len(class_name))

model_0.to('cpu')

FashionMNISTModel(
  (layer_one): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
  )
)

## Loss and Optimitization metrics

In [58]:
import requests
from pathlib import Path

In [63]:
#check validity of url

try:

    r = requests.get('https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')
    print(f"{r.status_code}")

    if r.status_code == 200:
        if Path('helper_functions.py').is_file():
            print(f" The files already exist now...Skipping download..")

        else:
            with open('helper_functions.py', 'wb') as file:
                print('files are being downloaded')

            file.write(r.content)
except Exception as e:
    print(" Unkown error")



200
 The files already exist now...Skipping download..


In [64]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr= 0.01)

the data in our case is in batches and we need two loops , one for epochs and the other one for batches

In [ ]:
#import tqdm for visualization of the progressbar as we train the model
from tqdm.auto import tqdm

In [70]:
#training loop
epochs = 3

for epoch in tqdm(range(epochs)):

    print(f"This is the epoch: {epoch}")

    training_loss = 0

    for Batch, (X,Y) in enumerate(training_data_batch):

        model_0.train()
        
        #forward pass
        y_pred = model_0(X)

        #calculate the loss

        pred_loss = loss_fn(y_pred, Y)

        training_loss += pred_loss

        #optimizer and zero grad

        optimizer.zero_grad()

        #step backwards
        pred_loss.backward()

        #optimizer step
        optimizer.step()

        if Batch % 700 == 0:
            print(f"The amount of data processed is {Batch*len(X)}")




  0%|          | 0/3 [00:00<?, ?it/s]

This is the epoch: 0
The amount of data processed is 0
The amount of data processed is 22400
The amount of data processed is 44800
This is the epoch: 1
The amount of data processed is 0
The amount of data processed is 22400
The amount of data processed is 44800
This is the epoch: 2
The amount of data processed is 0
The amount of data processed is 22400
The amount of data processed is 44800
